##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Loop de treinamento personalizado com o Keras e MultiWorkerMirroredStrategy

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_ctl"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/distribute/multi_worker_with_ctl.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

## Visão geral

Este tutorial demonstra como fazer um treinamento distribuído multiworker com um modelo do Keras e com [loops de treinamento personalizado](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) usando a API `tf.distribute.Strategy`. O loop de treinamento é distribuído pelo `tf.distribute.MultiWorkerMirroredStrategy` de tal modo que um modelo `tf.keras` — desenvolvido para executar em um [único worker](custom_training.ipynb) — possa funcionar de forma transparente em diversos workers, com alterações de código mínimas. Os loops de treinamento personalizado proporcionam flexibilidade e um maior controle do treinamento, além de facilitar a depuração do modelo. Saiba mais sobre [como escrever um loop de treinamento básico](../../guide/basic_training_loops.ipynb), [como escrever um loop de treinamento do zero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) e [treinamento personalizado](../customization/custom_training_walkthrough.ipynb).

Se você deseja saber como usar `MultiWorkerMirroredStrategy` com `tf.keras.Model.fit`, confira este [tutorial](multi_worker_with_keras.ipynb).

O guia [Treinamento distribuído no TensorFlow](../../guide/distributed_training.ipynb) apresenta uma visão geral das estratégias de distribuição oferecidas pelo TensorFlow para quem tiver interesse em compreender de forma mais aprofundada as APIs de `tf.distribute.Strategy`.

## Configuração

Primeiro, faça as importações necessárias.

In [ ]:
import json
import os
import sys

Antes de importar o TensorFlow, faça algumas alterações no ambiente:

- Desative todas as GPUs. Isso evita erros causados quando todos os workers tentam usar a mesma GPU. Em uma aplicação real, cada worker estaria em uma máquina diferente.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

- Redefina a variável de ambiente `'TF_CONFIG'` (você verá mais detalhes sobre isso posteriormente).

In [ ]:
os.environ.pop('TF_CONFIG', None)

- O diretório atual deve estar no caminho do Python. Dessa forma, o notebook pode importar os arquivos escritos por `%%writefile` posteriormente.


In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Agora, importe o TensorFlow.

In [ ]:
import tensorflow as tf

### Definição do dataset e do modelo

Agora, crie um arquivo `mnist.py` com uma configuração simples de modelo e dataset. Este arquivo do Python será usado pelos processos dos workers neste tutorial:

In [ ]:
%%writefile mnist.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # The `x` arrays are in uint8 and have values in the range [0, 255].
  # You need to convert them to float32 with values in the range [0, 1]
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000)
  return train_dataset

def dataset_fn(global_batch_size, input_context):
  batch_size = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = mnist_dataset(batch_size)
  dataset = dataset.shard(input_context.num_input_pipelines,
                          input_context.input_pipeline_id)
  dataset = dataset.batch(batch_size)
  return dataset

def build_cnn_model():
  regularizer = tf.keras.regularizers.L2(1e-5)
  return tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3,
                             activation='relu',
                             kernel_regularizer=regularizer),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128,
                            activation='relu',
                            kernel_regularizer=regularizer),
      tf.keras.layers.Dense(10, kernel_regularizer=regularizer)
  ])

## Configuração multiworker

Agora, vamos entrar na seara do treinamento multiworker. No TensorFlow, a variável de ambiente `'TF_CONFIG'` é necessária para treinamento em diversas máquinas. Cada máquina pode ter uma função diferente. A variável `'TF_CONFIG'` usada abaixo é uma string JSON que especifica a configuração de cluster em cada worker que faz parte do cluster. Esse é o método padrão de especificação de um cluster, usando `cluster_resolver.TFConfigClusterResolver`, mas existem outras opções disponíveis no módulo `distribute.cluster_resolver`. Saiba mais sobre como configurar a variável `'TF_CONFIG'` no guia [Treinamento distribuído](../../guide/distributed_training.ipynb).

### Descrição do cluster

Veja um exemplo de configuração:

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

Observe que `tf_config` é apenas uma variável local no Python. Para usá-la na configuração do treinamento, serialize-a como JSON e coloque-a em uma variável de ambiente `'TF_CONFIG'`. Veja a mesma variável `'TF_CONFIG'` serializada como string JSON:

In [ ]:
json.dumps(tf_config)

Existem dois componentes de `'TF_CONFIG'`: `'cluster'` e `'task'` (tarefa).

- `'cluster'` é igual para todos os workers e fornece informações sobre o cluster de treinamento, que é um dicionário consistindo de diferentes tipos de trabalhos, como `'worker'`. No treinamento multiworker com `MultiWorkerMirroredStrategy`, geralmente existe um `'worker'` que tem mais responsabilidades, como salvar checkpoints e escrever arquivos de resumo para o TensorBoard, além das tarefas de um `'worker'` comum. Esse tipo de worker é chamado de worker `'chief'` (principal), e geralmente o `'worker'` com `'index'` (índice) 0 é designado como o `worker` principal.

- `'task'` fornece informações da tarefa atual e é diferente em cada worker. Ela especifica o `'type'` (tipo) e o `'index'` (índice) desse worker.

Neste exemplo, você define o `'type'` da tarefa como `'worker'` e o `'index'` da tarefa como `0`. Esta máquina é o primeiro worker e será designada como o worker principal, realizando mais trabalho do que os outros. Também será preciso definir a variável de ambiente `'TF_CONFIG'` nas outras máquinas, e ela deve ter o mesmo dicionário `'cluster'`, mas um `'type'` de tarefa ou um `'index'` de tarefa diferente, dependendo das funções das máquinas.


Para fins de ilustração, este tutorial mostra como definir uma variável `'TF_CONFIG'` com dois workers no `'localhost'`.  Na prática, os usuários criariam diversos workers em endereços IP/portas externos e definiriam `'TF_CONFIG'` em cada worker da forma adequada.

Este exemplo usa dois workers. A variável `'TF_CONFIG'` do primeiro worker é exibida acima. Para o segundo worker, defina `tf_config['task']['index']=1`.

### Variáveis de ambiente e subprocessos nos notebooks

Os subprocessos herdam as variáveis de ambiente do pai. Portanto, se você definir uma variável de ambiente neste processo do Jupyter Notebook:

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

...você poderá acessar a variável de ambiente em um subprocesso:

In [ ]:
%%bash
echo ${GREETINGS}

Na próxima seção, você usará esse método para passar a variável `'TF_CONFIG'` aos subprocessos do worker. Você jamais iniciaria seus trabalhos dessa forma, mas é suficiente para as finalidades deste tutorial, para demonstrar um exemplo mínimo de multiworker.

## MultiWorkerMirroredStrategy

Antes de treinar o modelo, primeiro crie uma instância de `tf.distribute.MultiWorkerMirroredStrategy`:

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Observação: `'TF_CONFIG'` é processada e os servidores gRPC do TensorFlow são iniciados no momento em que você faz a chamada a `tf.distribute.MultiWorkerMirroredStrategy`. Portanto, você precisa definir a variável de ambiente `'TF_CONFIG'` antes de instanciar um `tf.distribute.Strategy`. Para economizar tempo neste exemplo ilustrativo, isso não é demonstrado neste tutorial para que não seja preciso iniciar os servidores. Um exemplo completo está disponível na última seção deste tutorial.

Use `tf.distribute.Strategy.scope` para especificar que uma estratégia deve ser usada ao criar o modelo. Dessa forma, a estratégia poderá controlar várias coisas, como o posicionamento de variáveis: ela criará cópias de todas as variáveis nas camadas do modelo em cada dispositivo, em todos os workers.

In [ ]:
import mnist
with strategy.scope():
  # Model building needs to be within `strategy.scope()`.
  multi_worker_model = mnist.build_cnn_model()

## Fragmentação automática dos dados nos workers

No treinamento multiworker, a *fragmentação do dataset* é necessária para convergência e capacidade de reprodutibilidade. Com a fragmentação, cada worker recebe um subconjunto do dataset, o que ajuda a criar uma experiência similar ao treinamento em um único worker. No exemplo abaixo, é utilizada a política padrão de fragmentação automática de `tf.distribute`. Você também pode personalizá-la definindo a `tf.data.experimental.AutoShardPolicy` de `tf.data.experimental.DistributeOptions`. Para saber mais, confira a seção *Fragmentação* do [tutorial Entrada distribuída](input.ipynb).

In [ ]:
per_worker_batch_size = 64
num_workers = len(tf_config['cluster']['worker'])
global_batch_size = per_worker_batch_size * num_workers

with strategy.scope():
  multi_worker_dataset = strategy.distribute_datasets_from_function(
      lambda input_context: mnist.dataset_fn(global_batch_size, input_context))

## Definição de um loop de treinamento personalizado e treinamento do modelo

Especifique um otimizador:

In [ ]:
with strategy.scope():
  # The creation of optimizer and train_accuracy needs to be in
  # `strategy.scope()` as well, since they create variables.
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')

Defina um passo de treinamento com `tf.function`:


In [ ]:
@tf.function
def train_step(iterator):
  """Training step function."""

  def step_fn(inputs):
    """Per-Replica step function."""
    x, y = inputs
    with tf.GradientTape() as tape:
      predictions = multi_worker_model(x, training=True)
      per_example_loss = tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True,
          reduction=tf.keras.losses.Reduction.NONE)(y, predictions)
      loss = tf.nn.compute_average_loss(per_example_loss)
      model_losses = multi_worker_model.losses
      if model_losses:
        loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))

    grads = tape.gradient(loss, multi_worker_model.trainable_variables)
    optimizer.apply_gradients(
        zip(grads, multi_worker_model.trainable_variables))
    train_accuracy.update_state(y, predictions)
    return loss

  per_replica_losses = strategy.run(step_fn, args=(next(iterator),))
  return strategy.reduce(
      tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

### Como salvar e restaurar checkpoints

Ao escrever um loop de treinamento personalizado, você precisa [salvar checkpoints](../../guide/checkpoint.ipynb) manualmente em vez de fazer um callback ao Keras. Para o `MultiWorkerMirroredStrategy`, salvar um checkpoint ou um modelo completo requer a participação de todos os workers, pois tentar salvar somente no worker principal poderia levar a um deadlock. Os workers também precisam escrever em caminhos diferentes para evitar sobrescrever os arquivos. Veja um exemplo de como configurar os diretórios:

In [ ]:
from multiprocessing import util
checkpoint_dir = os.path.join(util.get_temp_dir(), 'ckpt')

def _is_chief(task_type, task_id, cluster_spec):
  return (task_type is None
          or task_type == 'chief'
          or (task_type == 'worker'
              and task_id == 0
              and "chief" not in cluster_spec.as_dict()))

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id, cluster_spec):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id, cluster_spec):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

Crie um `tf.train.Checkpoint` que monitore o modelo, que é gerenciado por um `tf.train.CheckpointManager` de forma que apenas os últimos checkpoints sejam preservados:

In [ ]:
epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64), name='epoch')
step_in_epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64),
    name='step_in_epoch')
task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
# Normally, you don't need to manually instantiate a `ClusterSpec`, but in this
# illustrative example you did not set `'TF_CONFIG'` before initializing the
# strategy. Check out the next section for "real-world" usage.
cluster_spec = tf.train.ClusterSpec(tf_config['cluster'])

checkpoint = tf.train.Checkpoint(
    model=multi_worker_model, epoch=epoch, step_in_epoch=step_in_epoch)

write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id,
                                      cluster_spec)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

Agora, quando você precisar restaurar um checkpoint, encontrará o último checkpoint salvo usando a função conveniente `tf.train.latest_checkpoint` (ou fazendo uma chamada a `tf.train.CheckpointManager.restore_or_initialize`).

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
  checkpoint.restore(latest_checkpoint)

Após restaurar o checkpoint, você pode continuar treinando o seu loop de treinamento personalizado.

In [ ]:
num_epochs = 3
num_steps_per_epoch = 70

while epoch.numpy() < num_epochs:
  iterator = iter(multi_worker_dataset)
  total_loss = 0.0
  num_batches = 0

  while step_in_epoch.numpy() < num_steps_per_epoch:
    total_loss += train_step(iterator)
    num_batches += 1
    step_in_epoch.assign_add(1)

  train_loss = total_loss / num_batches
  print('Epoch: %d, accuracy: %f, train_loss: %f.'
                %(epoch.numpy(), train_accuracy.result(), train_loss))

  train_accuracy.reset_states()

  # Once the `CheckpointManager` is set up, you're now ready to save, and remove
  # the checkpoints non-chief workers saved.
  checkpoint_manager.save()
  if not _is_chief(task_type, task_id, cluster_spec):
    tf.io.gfile.rmtree(write_checkpoint_dir)

  epoch.assign_add(1)
  step_in_epoch.assign(0)

## Confira o código completo

Resumindo todos os procedimentos abordados até agora:

1. Crie processos nos workers.
2. Passe variáveis `'TF_CONFIG'` aos processos dos workers.
3. Deixe cada processo dos workers executar o script abaixo, que contém o código de treinamento.

In [ ]:
%%writefile main.py
#@title File: `main.py`
import os
import json
import tensorflow as tf
import mnist
from multiprocessing import util

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])
global_batch_size = per_worker_batch_size * num_workers

num_epochs = 3
num_steps_per_epoch=70

# Checkpoint saving and restoring
def _is_chief(task_type, task_id, cluster_spec):
  return (task_type is None
          or task_type == 'chief'
          or (task_type == 'worker'
              and task_id == 0
              and 'chief' not in cluster_spec.as_dict()))

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id, cluster_spec):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id, cluster_spec):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

checkpoint_dir = os.path.join(util.get_temp_dir(), 'ckpt')

# Define Strategy
strategy = tf.distribute.MultiWorkerMirroredStrategy()

with strategy.scope():
  # Model building/compiling need to be within `tf.distribute.Strategy.scope`.
  multi_worker_model = mnist.build_cnn_model()

  multi_worker_dataset = strategy.distribute_datasets_from_function(
      lambda input_context: mnist.dataset_fn(global_batch_size, input_context))
  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')

@tf.function
def train_step(iterator):
  """Training step function."""

  def step_fn(inputs):
    """Per-Replica step function."""
    x, y = inputs
    with tf.GradientTape() as tape:
      predictions = multi_worker_model(x, training=True)
      per_example_loss = tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True,
          reduction=tf.keras.losses.Reduction.NONE)(y, predictions)
      loss = tf.nn.compute_average_loss(per_example_loss)
      model_losses = multi_worker_model.losses
      if model_losses:
        loss += tf.nn.scale_regularization_loss(tf.add_n(model_losses))

    grads = tape.gradient(loss, multi_worker_model.trainable_variables)
    optimizer.apply_gradients(
        zip(grads, multi_worker_model.trainable_variables))
    train_accuracy.update_state(y, predictions)

    return loss

  per_replica_losses = strategy.run(step_fn, args=(next(iterator),))
  return strategy.reduce(
      tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64), name='epoch')
step_in_epoch = tf.Variable(
    initial_value=tf.constant(0, dtype=tf.dtypes.int64),
    name='step_in_epoch')

task_type, task_id, cluster_spec = (strategy.cluster_resolver.task_type,
                                    strategy.cluster_resolver.task_id,
                                    strategy.cluster_resolver.cluster_spec())

checkpoint = tf.train.Checkpoint(
    model=multi_worker_model, epoch=epoch, step_in_epoch=step_in_epoch)

write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id,
                                      cluster_spec)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

# Restoring the checkpoint
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
  checkpoint.restore(latest_checkpoint)

# Resume our CTL training
while epoch.numpy() < num_epochs:
  iterator = iter(multi_worker_dataset)
  total_loss = 0.0
  num_batches = 0

  while step_in_epoch.numpy() < num_steps_per_epoch:
    total_loss += train_step(iterator)
    num_batches += 1
    step_in_epoch.assign_add(1)

  train_loss = total_loss / num_batches
  print('Epoch: %d, accuracy: %f, train_loss: %f.'
                %(epoch.numpy(), train_accuracy.result(), train_loss))

  train_accuracy.reset_states()

  checkpoint_manager.save()
  if not _is_chief(task_type, task_id, cluster_spec):
    tf.io.gfile.rmtree(write_checkpoint_dir)

  epoch.assign_add(1)
  step_in_epoch.assign(0)

Agora, o diretório atual contém os dois arquivos do Python:

In [ ]:
%%bash
ls *.py

Então, serialize a variável `'TF_CONFIG'` como JSON e adicione-a às variáveis de ambiente:

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Agora, você pode iniciar um processo de worker, que executará `main.py` e usará a variável `'TF_CONFIG'`:

In [ ]:
# first kill any previous runs
%killbgscripts

In [ ]:
%%bash --bg
python main.py &> job_0.log

Veja algumas observações sobre o comando acima:

1. Ele usa `%%bash`, que é um [“magic” dos notebooks](https://ipython.readthedocs.io/en/stable/interactive/magics.html) que executa alguns comandos bash.
2. Ele usa o sinalizador `--bg` para executar o processo `bash` em segundo plano, pois esse worker não será encerrado. Ele aguarda todos os workers antes de iniciar.

O processo do worker em segundo plano não exibe a saída nesse notebook. `&>` redireciona a saída para um arquivo para que você possa analisar o que aconteceu.

Aguarde alguns segundos para o processo iniciar:

In [ ]:
import time
time.sleep(20)

Agora, verifique a saída no arquivo de log do worker:

In [ ]:
%%bash
cat job_0.log

A última linha do arquivo de log deve dizer: `Started server with target: grpc://localhost:12345` (Serviço iniciado com destino: grpc://localhost:12345). Agora, o primeiro worker está pronto e aguarda que todos os outros workers estejam prontos para prosseguir.

Atualize `tf_config` para o processo do segundo worker prosseguir:

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Agora, inicie o segundo worker. O treinamento será iniciado, já que todos os workers estão ativos (então não há necessidade de colocar esse processo em segundo plano):

In [ ]:
%%bash
python main.py > /dev/null 2>&1

Se você verificar novamente os logs escritos pelo primeiro worker, notará que ele participou do treinamento do modelo:

In [ ]:
%%bash
cat job_0.log

In [ ]:
# Delete the `'TF_CONFIG'`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

## Treinamento multiworker aprofundado

Este tutorial demonstrou um fluxo de trabalho de loop de treinamento personalizado para a configuração multiworker. Estão disponíveis descrições detalhadas de outros tópicos no tutorial [Treinamento multiworker com o Keras (`tf.keras.Model.fit`)](multi_worker_with_keras.ipynb), aplicável a loops de treinamento personalizado.

## Saiba mais

1. O guia [Treinamento distribuído no TensorFlow](../../guide/distributed_training.ipynb) apresenta uma visão geral das estratégias de distribuição disponíveis.
2. [Modelos oficiais](https://github.com/tensorflow/models/tree/master/official), muitos dos quais podem ser configurados para executarem diversas estratégias de distribuição.
3. A [seção Desempenho](../../guide/function.ipynb) do guia do `tf.function` apresenta informações sobre outras estratégias e [ferramentas](../../guide/profiler.md) que você pode usar para otimizar o desempenho dos seus modelos do TensorFlow.
